### Week 22 In-Class Assignment
#### Angela Spencer - February 23, 2022

#### 1. Use Splinter and/or Beautiful Soup to scrape mercadolibre.com.mx or soundcloud.com or tripadvisor.com. You must programmatically click a link from the front page to go to another page of interest to you and then select at least 5 items from that page. The 5 items must be stored in a data structure that makes sense (dictionary, list, etc).

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [56]:
executable_path = {'executable_path':ChromeDriverManager().install()}

browser = Browser('chrome', **executable_path, headless=False)




====== WebDriver manager ======
Current google-chrome version is 98.0.4758
Get LATEST chromedriver version for 98.0.4758 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\aspen\.wdm\drivers\chromedriver\win32\98.0.4758.102]


In [93]:
#direct browser to home page
url = 'https://soundcloud.com/'
browser.visit(url)

#direct browser to click on link containing 'Explore' with a wait period before continuing
browser.links.find_by_partial_text('Explore').click()
time.sleep(1)

#parse html from current url
soup = bs(browser.html, 'html.parser')

#extract all div labels with specified class to access each music chart tile
all_divs = soup.find_all('div', class_='playableTile__artwork')

#for each div, extract chart, link, and image info
soundcloud_list = []
for div in all_divs[:5]:
    try:
        chart = div.find('span')['aria-label']
        print(chart)
        
        link = div.find('a')['href']
        print(link)
        
        image = re.search('http.*jpg', div.find('span')['style']).group(0)
        print(image)
        
# add info to a dictionary and append dictionary to empty list        
        chart_dict = {'chart_name':chart,
                      'reference_url':link, 
                      'image_link':image}

        soundcloud_list.append(chart_dict)
        
    except Exception as e:
        print(e)

Top 50: All music genres
/discover/sets/charts-top:all-music:us
https://i1.sndcdn.com/artworks-3MZHy2YEhMd3-0-t200x200.jpg
Top 50: Hip-hop & Rap
/discover/sets/charts-top:hiphoprap:us
https://i1.sndcdn.com/artworks-3MZHy2YEhMd3-0-t200x200.jpg
Top 50: Pop
/discover/sets/charts-top:pop:us
https://i1.sndcdn.com/artworks-teTQbwTgfaES-0-t200x200.jpg
Top 50: R&B & Soul
/discover/sets/charts-top:rbsoul:us
https://i1.sndcdn.com/artworks-9ixbhhfjgJ9wWsGI-TlItkQ-t200x200.jpg
Top 50: Global Beats
/discover/sets/charts-top:world:us
https://i1.sndcdn.com/artworks-000350395503-2duk8l-t200x200.jpg


In [94]:
#convert list of dictionaries to a dataframe
soundcloud_df = pd.DataFrame(soundcloud_list)
soundcloud_df

,chart_name,reference_url,image_link
0,Top 50: All music genres,/discover/sets/charts-top:all-music:us,https://i1.sndcdn.com/artworks-3MZHy2YEhMd3-0-...
1,Top 50: Hip-hop & Rap,/discover/sets/charts-top:hiphoprap:us,https://i1.sndcdn.com/artworks-3MZHy2YEhMd3-0-...
2,Top 50: Pop,/discover/sets/charts-top:pop:us,https://i1.sndcdn.com/artworks-teTQbwTgfaES-0-...
3,Top 50: R&B & Soul,/discover/sets/charts-top:rbsoul:us,https://i1.sndcdn.com/artworks-9ixbhhfjgJ9wWsG...
4,Top 50: Global Beats,/discover/sets/charts-top:world:us,https://i1.sndcdn.com/artworks-000350395503-2d...


#### 2. Share with your group how web scraping worked on the website and link you chose. Compare it with others. List the site you chose and two observations about parsing the website. Indicate if this was the same or different for the sites chosen by others.

We used the soundcloud.com website and pulled the first 5 charts from the 'Discover' page.
One difficulty here was extracting information from the 'aria-label':
- < span aria-label="Top 50: All music genres" ...
The syntax here was to call on aria-label as though it were an index inside of span:
- div.find('span')['aria-label']
There was no need to use .text in this case.

Another difficulty was extracting a url embedded in a larger string:
- 'background-image: url("https://i1.sndcdn.com/artworks-3MZHy2YEhMd3-0-t200x200.jpg"); width: 100%; height: 100%; opacity: 0;'
The only way to access this was with regex syntax:
- re.search('http.*jpg', div.find('span')['style']).group(0)
    
Once we discovered these syntax secrets, we were able to pull all desired information from the web page.